# Install Dependencies
Install additional packages needed to run SQL command in Jupyter cells and send them to a Postgres database server.

## PostgreSQL
Allows for a self-contained instance of PostresSQL for this notebook.

In [ ]:
%mamba install -y -c conda-forge postgresql=17 --quiet
!psql --version

## PostgreSQL Python Extension

In [ ]:
%mamba install -y -c conda-forge postgresql-plpython=17 --quiet

## PostgreSQL Adapter for Python
Essential for connecting and interacting with PostgreSQL databases from Python code in Jupyter notebooks.

In [ ]:
%mamba install -c conda-forge psycopg2 -y --quiet
!python -c "import psycopg2; print(psycopg2.__version__)"

## Language Server Protocol for Jupyter
Enables code completion, hover documentation, and real-time syntax checking for multiple languages in Jupyter notebooks, including SQL.

In [ ]:
%mamba install -c conda-forge jupyterlab-lsp -y --quiet
!jupyter labextension list
!jupyter server extension list

## Language Server for Python
Provides real-time linting, code suggestions, and auto-completion in Jupyter notebooks for Python code.

In [ ]:
%mamba install -c conda-forge python-lsp-server -y --quiet
!pylsp --version

## SQL Magic Extension for Jupyter
Allows running SQL queries directly in Jupyter notebooks using SQL magic (%%sql). Enables seamless execution of SQL queries inside a Jupyter Notebook without switching to a database client.

In [ ]:
%mamba install -c conda-forge jupysql -y --quiet
!python -c "import sql; print(sql.__version__)"
!jupyter labextension list

## SQL Editor Extension
Enables SQL-specific features in Jupyter notebooks, such as:
* Syntax highlighting
* Query auto-completion
* Error checking

In [ ]:
%pip install jupyterlab_sql_editor --quiet
!jupyter labextension list

# Start PostgreSQL

## Define paths and logfile

In [ ]:
import os

DB_DIR = "mylocal_db"
LOGFILE = "postgres_log.txt"
PG_HBA_PATH = os.path.join(DB_DIR, "pg_hba.conf")

## Check PostgreSQL status

In [ ]:
!pg_ctl -D $DB_DIR status

## Stop any existing PostgreSQL process

In [ ]:
!pg_ctl stop -D $DB_DIR -m fast || echo "Nothing to stop."

## Initialize the database if necessary

In [ ]:
import os

if not os.path.exists(DB_DIR) or not os.path.exists(os.path.join(DB_DIR, "PG_VERSION")):
    os.makedirs(DB_DIR, exist_ok=True)
    !initdb -D $DB_DIR

## Modify pg_hba.conf to allow trust authentication for user "notebook"

In [ ]:
with open(PG_HBA_PATH, "a") as pg_hba:
    pg_hba.write("\nlocal   all             notebook                                trust\n")
    pg_hba.write("host    all             notebook        127.0.0.1/32          trust\n")
    pg_hba.write("host    all             notebook        ::1/128               trust\n")

## Start PostgreSQL server

In [ ]:
import subprocess
import time
import atexit
import sys
import os

# Define paths
conda_prefix = sys.prefix
pg_ctl_path = os.path.join(conda_prefix, 'bin', 'pg_ctl')

def stop_postgres():
    subprocess.run([pg_ctl_path, "-D", DB_DIR, "stop"])

atexit.register(stop_postgres)

with open(LOGFILE, "a") as logfile:
    subprocess.Popen([pg_ctl_path, "-D", DB_DIR, "-l", LOGFILE, "start"],
                     stdout=logfile, stderr=logfile)

time.sleep(5)

## Create the "notebook" user with full permissions

In [ ]:
!psql -d postgres -c "CREATE USER notebook WITH SUPERUSER;" || echo "User already exists."
!psql -d postgres -c "ALTER USER notebook WITH PASSWORD NULL;"

# Connect

## Load & Configure SQLMagic

In [ ]:
%load_ext sql
%config SqlMagic.displaylimit = None

## Create Fresh sahuagin Database

In [ ]:
!psql -U notebook -d postgres -c "SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname = 'sahuagin';"
!psql -U notebook -d postgres -c "DROP DATABASE IF EXISTS sahuagin;"
!psql -U notebook -d postgres -c "CREATE DATABASE sahuagin;"

## Connect to sahuagin Database

In [ ]:
%sql postgresql://notebook@localhost:5432/sahuagin

# Create Tables & Types

In [ ]:
%%sql

CREATE EXTENSION IF NOT EXISTS plpython3u;
CREATE EXTENSION IF NOT EXISTS citext;

-- Mechanism table replaces activable, variant, variable, and space.
-- A mechanism is defined by its programming language and serialized code.
-- It must have a unique (non-null) name.
DROP TABLE IF EXISTS mechanism CASCADE;
CREATE TABLE mechanism (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  name citext NOT NULL UNIQUE,
  serialized text NOT NULL
);

-- Activation table now links mechanisms.
-- "from_mechanism" continues through an activation (by name) to the "to_mechanism",
-- and the activation is defined in the context of a "root_mechanism".
-- The unique constraint ensures that (from_mechanism, root_mechanism, name) is unique.
DROP TABLE IF EXISTS activation CASCADE;
CREATE TABLE activation (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  name citext NOT NULL,
  from_mechanism integer NOT NULL,
  root_mechanism integer NOT NULL,
  to_mechanism integer NOT NULL,
  CONSTRAINT uq_activation UNIQUE (from_mechanism, root_mechanism, name),
  CONSTRAINT fk_activation_from FOREIGN KEY (from_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE,
  CONSTRAINT fk_activation_root FOREIGN KEY (root_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE,
  CONSTRAINT fk_activation_to FOREIGN KEY (to_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE
);

-- New table "unmasking"
-- This table provides a reference between a root mechanism, an activation,
-- and a mechanism that is being unmasked (unmasked_to_mechanism).
DROP TABLE IF EXISTS unmasking CASCADE;
CREATE TABLE unmasking (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  root_mechanism integer NOT NULL,
  activation integer NOT NULL,
  unmasked_to_mechanism integer NOT NULL,
  CONSTRAINT fk_unmasking_root FOREIGN KEY (root_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE,
  CONSTRAINT fk_unmasking_activation FOREIGN KEY (activation)
    REFERENCES activation(id) ON DELETE CASCADE,
  CONSTRAINT fk_unmasking_unmasked FOREIGN KEY (unmasked_to_mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE
);

-- The entity table now associates an entity with a mechanism.
DROP TABLE IF EXISTS entity CASCADE;
CREATE TABLE entity (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  name citext NOT NULL,
  mechanism integer NOT NULL,
  CONSTRAINT fk_entity_mechanism FOREIGN KEY (mechanism)
    REFERENCES mechanism(id) ON DELETE CASCADE
);

-- Snapshot (state) of an entity at a moment in time.
DROP TABLE IF EXISTS state CASCADE;
CREATE TABLE state (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  entity integer NOT NULL,
  time double precision NOT NULL,
  CONSTRAINT uq_state UNIQUE (entity, time),
  CONSTRAINT fk_state_entity FOREIGN KEY (entity)
    REFERENCES entity(id) ON DELETE CASCADE
);

-- Locked relationships between state and activation for partial re-generation.
DROP TABLE IF EXISTS locked_activation CASCADE;
CREATE TABLE locked_activation (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  state integer NOT NULL,
  activation integer NOT NULL,
  CONSTRAINT fk_locked_activation_state FOREIGN KEY (state)
    REFERENCES state(id) ON DELETE CASCADE,
  CONSTRAINT fk_locked_activation_activation FOREIGN KEY (activation)
    REFERENCES activation(id) ON DELETE CASCADE
);

-- Enum indicating the type of value stored.
DROP TYPE IF EXISTS value_type CASCADE;
CREATE TYPE value_type AS ENUM ('string', 'number');

-- Abstract value representing mechanism states.
DROP TABLE IF EXISTS value CASCADE;
CREATE TABLE value (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  state integer NOT NULL,
  activation integer NOT NULL,
  name citext NOT NULL,
  type value_type NOT NULL,
  CONSTRAINT fk_value_state FOREIGN KEY (state)
    REFERENCES state(id) ON DELETE CASCADE,
  CONSTRAINT fk_value_activation FOREIGN KEY (activation)
    REFERENCES activation(id) ON DELETE CASCADE
);

-- Numeric value.
DROP TABLE IF EXISTS number_value CASCADE;
CREATE TABLE number_value (
  value integer PRIMARY KEY,
  serialized double precision NOT NULL,
  CONSTRAINT fk_number_value FOREIGN KEY (value)
    REFERENCES value(id) ON DELETE CASCADE
);

-- String value.
DROP TABLE IF EXISTS string_value CASCADE;
CREATE TABLE string_value (
  value integer PRIMARY KEY,
  serialized text NOT NULL,
  CONSTRAINT fk_string_value FOREIGN KEY (value)
    REFERENCES value(id) ON DELETE CASCADE
);

DROP TABLE IF EXISTS value_antecedent CASCADE;
CREATE TABLE value_antecedent (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    value integer NOT NULL,
    child integer NOT NULL,
    antecedent integer NOT NULL,
    CONSTRAINT fk_value_antecedent_value FOREIGN KEY (value)
        REFERENCES value(id) ON DELETE CASCADE,
    CONSTRAINT fk_value_antecedent_child FOREIGN KEY (child)
        REFERENCES activation(id) ON DELETE CASCADE,
    CONSTRAINT fk_value_antecedent_antecedent FOREIGN KEY (antecedent)
        REFERENCES activation(id) ON DELETE CASCADE,
    UNIQUE (value, child, antecedent)
);

DROP TABLE IF EXISTS locked_dependency CASCADE;
CREATE TABLE locked_dependency (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    state integer NOT NULL,
    value integer NOT NULL,
    activation integer NOT NULL,
    CONSTRAINT fk_locked_dependency_state FOREIGN KEY (state)
        REFERENCES state(id) ON DELETE CASCADE,
    CONSTRAINT fk_locked_dependency_value FOREIGN KEY (value)
        REFERENCES value(id) ON DELETE CASCADE,
    CONSTRAINT fk_locked_dependency_activation FOREIGN KEY (activation)
        REFERENCES activation(id) ON DELETE CASCADE,
    UNIQUE (state, value, activation)
);

-- grouping of entity states.
DROP TABLE IF EXISTS grouping CASCADE;
CREATE TABLE grouping (
  id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
  name citext NOT NULL UNIQUE
);

-- Entity observed in a grouping.
DROP TABLE IF EXISTS grouping_entity CASCADE;
CREATE TABLE grouping_entity (
  entity integer PRIMARY KEY,
  grouping integer NOT NULL,
  CONSTRAINT fk_grouping_entity_entity FOREIGN KEY (entity)
    REFERENCES entity(id) ON DELETE CASCADE,
  CONSTRAINT fk_grouping_entity_grouping FOREIGN KEY (grouping)
    REFERENCES grouping(id) ON DELETE CASCADE
);

# Debug

## Create Log Table

In [ ]:
%%sql

DROP TABLE IF EXISTS debug_log CASCADE;
CREATE TABLE debug_log (
    id integer GENERATED ALWAYS AS IDENTITY PRIMARY KEY,
    log_time timestamp DEFAULT CURRENT_TIMESTAMP,
    procedure_name varchar(255),
    log_message text
);

## Create Log Function

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION debug_log(
    p_procedure_name varchar,
    p_log_message text
) RETURNS void
LANGUAGE plpgsql
AS $$
BEGIN
    INSERT INTO debug_log (procedure_name, log_message, log_time)
    VALUES (p_procedure_name, p_log_message, now());
END;
$$;

## Clear Log Table

In [ ]:
%%sql

TRUNCATE TABLE debug_log RESTART IDENTITY CASCADE;

In [ ]:
%%sql

SELECT * FROM debug_log;

# Functions/Procedures

## Get Activation Full Path

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_activation_full_path(activation_id integer)
RETURNS text
LANGUAGE plpgsql
STABLE
AS $$
DECLARE
    full_path text;
BEGIN
    WITH RECURSIVE act_path AS (
        -- Start with the given activation.
        SELECT 
            id,
            name,
            from_mechanism,
            root_mechanism,
            to_mechanism,
            name AS full_path
        FROM activation
        WHERE id = activation_id

        UNION ALL

        -- Recursively join parent's activation where parent's to_mechanism
        -- matches the child's from_mechanism and they share the same root.
        SELECT 
            p.id,
            p.name,
            p.from_mechanism,
            p.root_mechanism,
            p.to_mechanism,
            p.name || '/' || ap.full_path AS full_path
        FROM activation p
        JOIN act_path ap 
          ON p.to_mechanism = ap.from_mechanism 
         AND p.root_mechanism = ap.root_mechanism
    )
    -- The root activation in the chain will have no parent – i.e. no activation
    -- exists such that its to_mechanism equals this activation's from_mechanism.
    SELECT full_path INTO full_path
    FROM act_path
    WHERE NOT EXISTS (
        SELECT 1 
        FROM activation p2
        WHERE p2.to_mechanism = act_path.from_mechanism 
          AND p2.root_mechanism = act_path.root_mechanism
    )
    LIMIT 1;
    
    RETURN full_path;
END;
$$;

## Check Unmasking

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION check_unmasking(
    p_root_mechanism INTEGER,
    p_parent_activation_path TEXT,
    p_local_activation_name TEXT
) RETURNS INTEGER AS $$
DECLARE
    effective_key TEXT;
    candidate RECORD;
BEGIN
    -- Compute the full activation path for the new activation.
    IF p_parent_activation_path IS NULL OR p_parent_activation_path = '' THEN
        effective_key := p_local_activation_name;
    ELSE
        effective_key := p_parent_activation_path || '/' || p_local_activation_name;
    END IF;

    /*
      For each unmasking record for the given root mechanism,
      compute a candidate effective key by calling the helper
      get_activation_full_path on the stored activation and then:
         candidate_key = (p_parent_activation_path || '/' || get_activation_full_path(u.activation))
         (or just get_activation_full_path(u.activation) if no parent exists)
      Then select the record whose candidate key exactly matches the effective_key.
      If multiple match, pick the one with the deepest relative path.
    */
    SELECT u.unmasked_to_mechanism,
           get_activation_full_path(u.activation) AS rel_path,
           array_length(string_to_array(get_activation_full_path(u.activation), '/'), 1) AS depth
    INTO candidate
    FROM unmasking u
    WHERE u.root_mechanism = p_root_mechanism
      AND (
           CASE 
             WHEN p_parent_activation_path IS NULL OR p_parent_activation_path = '' 
             THEN get_activation_full_path(u.activation)
             ELSE p_parent_activation_path || '/' || get_activation_full_path(u.activation)
           END
         ) = effective_key
    ORDER BY array_length(string_to_array(get_activation_full_path(u.activation), '/'), 1) DESC
    LIMIT 1;

    IF candidate IS NULL THEN
       RETURN NULL;
    ELSE
       RETURN candidate.unmasked_to_mechanism;
    END IF;
END;
$$ LANGUAGE plpgsql;

## Create Mechanism

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE create_mechanism(
    p_mechanism_name CITEXT,
    p_serialized_code TEXT
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_mech_id INTEGER;
BEGIN
    INSERT INTO mechanism (name, serialized)
    VALUES (p_mechanism_name, p_serialized_code)
    RETURNING id INTO v_mech_id;
    
    RAISE NOTICE 'Mechanism "%" created with id %.', p_mechanism_name, v_mech_id;
EXCEPTION 
    WHEN unique_violation THEN
        RAISE EXCEPTION 'A mechanism with name "%" already exists.', p_mechanism_name;
END;
$$;


## Create Unmasking

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE create_unmasking(
    p_root_mechanism_name CITEXT,
    p_activation_path TEXT,
    p_unmasked_to_mechanism_name CITEXT
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_root_mech_id         INTEGER;
    v_unmasked_to_mech_id  INTEGER;
    v_activation_id        INTEGER;
BEGIN
    -- Lookup the root mechanism by its human-readable name.
    SELECT id INTO v_root_mech_id
      FROM mechanism
     WHERE name = p_root_mechanism_name;
    IF v_root_mech_id IS NULL THEN
        RAISE EXCEPTION 'No mechanism found with name "%" for root mechanism.', p_root_mechanism_name;
    END IF;

    -- Lookup the unmasked-to mechanism by its name.
    SELECT id INTO v_unmasked_to_mech_id
      FROM mechanism
     WHERE name = p_unmasked_to_mechanism_name;
    IF v_unmasked_to_mech_id IS NULL THEN
        RAISE EXCEPTION 'No mechanism found with name "%" for unmasked mechanism.', p_unmasked_to_mechanism_name;
    END IF;

    -- Retrieve the activation record using the helper function.
    SELECT a.id INTO v_activation_id
      FROM activation a
     WHERE get_activation_full_path(a.id) = p_activation_path
       AND a.root_mechanism = v_root_mech_id;
    IF v_activation_id IS NULL THEN
        RAISE EXCEPTION 'No activation found with path "%" under root mechanism "%".', p_activation_path, v_root_mech_id;
    END IF;

    -- Insert the unmasking record.
    INSERT INTO unmasking (root_mechanism, activation, unmasked_to_mechanism)
    VALUES (v_root_mech_id, v_activation_id, v_unmasked_to_mech_id);

    RAISE NOTICE 'Unmasking record created: root mechanism "%" | activation path "%" | unmasked to mechanism "%".',
      p_root_mechanism_name, p_activation_path, p_unmasked_to_mechanism_name;

    ----------------------------------------------------------------------------
    -- Delete entire state records for any entity whose state includes an activation
    -- (or descendant activations) that ever depended on the specified root mechanism.
    -- This avoids partial regeneration entirely.
    ----------------------------------------------------------------------------
    DELETE FROM state
    WHERE id IN (
        SELECT DISTINCT s.id
        FROM state s
        JOIN value v ON v.state = s.id
        JOIN activation a ON a.id = v.activation
        WHERE check_unmasking(
                  v_root_mech_id,
                  CASE 
                    WHEN strpos(get_activation_full_path(a.id), '/') > 0 
                      THEN substring(get_activation_full_path(a.id) from '^(.*)/')
                    ELSE ''
                  END,
                  a.name
              ) = v_unmasked_to_mech_id
    );
END;
$$;

## Lock Value

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE lock_value(
    p_entity_name VARCHAR,
    p_time DOUBLE PRECISION,
    p_activation_path VARCHAR,  -- Full activation path where the target value was produced.
    p_value_name VARCHAR        -- Name of the target value.
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_entity_id INTEGER;
    v_state_id INTEGER;
    v_value_id INTEGER;
BEGIN
    -- Look up the entity.
    SELECT id INTO v_entity_id FROM entity WHERE name = p_entity_name;
    IF v_entity_id IS NULL THEN
        RAISE EXCEPTION 'Entity "%" not found', p_entity_name;
    END IF;

    -- Look up the state.
    SELECT id INTO v_state_id FROM state WHERE entity = v_entity_id AND time = p_time;
    IF v_state_id IS NULL THEN
        RAISE EXCEPTION 'State for entity "%" at time % not found', p_entity_name, p_time;
    END IF;

    -- Find the target value (its id) produced by the given activation.
    SELECT v.id INTO v_value_id
    FROM value v
    JOIN activation a ON a.id = v.activation
    WHERE v.state = v_state_id
      AND v.name = p_value_name
      AND get_activation_full_path(a.id) = p_activation_path
    LIMIT 1;
    IF v_value_id IS NULL THEN
        RAISE EXCEPTION 'Value "%" not found in state with activation path %', p_value_name, p_activation_path;
    END IF;

    -- Compute full dependency closure for the target value.
    WITH RECURSIVE lock_chain(act) AS (
       -- Start with the activation that produced the target value.
       SELECT v.activation
       FROM value v
       WHERE v.state = v_state_id
         AND v.id = v_value_id
       UNION
       -- Walk upward: any activation that produced an input used by an activation in the chain.
       SELECT va.antecedent
       FROM value_antecedent va
       JOIN lock_chain lc ON va.child = lc.act
       UNION
       -- Walk downward: any activation that used an output from an activation in the chain.
       SELECT va.child
       FROM value_antecedent va
       JOIN lock_chain lc ON va.antecedent = lc.act
    )
    -- Record for each activation in the dependency closure that it is locked by this value.
    INSERT INTO locked_dependency(state, value, activation)
    SELECT v_state_id, v_value_id, act FROM lock_chain
    ON CONFLICT DO NOTHING;

    -- Ensure each activation in the closure is in locked_activation.
    INSERT INTO locked_activation(state, activation)
    SELECT v_state_id, act FROM lock_chain
    ON CONFLICT DO NOTHING;
END;
$$;

## Unlock Value

In [ ]:
CREATE OR REPLACE PROCEDURE unlock_value(
    p_entity_name VARCHAR,
    p_time DOUBLE PRECISION,
    p_activation_path VARCHAR,  -- Full activation path where the target value was produced.
    p_value_name VARCHAR        -- Name of the target value.
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_entity_id INTEGER;
    v_state_id INTEGER;
    v_value_id INTEGER;
BEGIN
    -- Look up the entity.
    SELECT id INTO v_entity_id FROM entity WHERE name = p_entity_name;
    IF v_entity_id IS NULL THEN
        RAISE EXCEPTION 'Entity "%" not found', p_entity_name;
    END IF;

    -- Look up the state.
    SELECT id INTO v_state_id FROM state WHERE entity = v_entity_id AND time = p_time;
    IF v_state_id IS NULL THEN
        RAISE EXCEPTION 'State for entity "%" at time % not found', p_entity_name, p_time;
    END IF;

    -- Find the target value (its id) produced by the given activation.
    SELECT v.id INTO v_value_id
    FROM value v
    JOIN activation a ON a.id = v.activation
    WHERE v.state = v_state_id
      AND v.name = p_value_name
      AND get_activation_full_path(a.id) = p_activation_path
    LIMIT 1;
    IF v_value_id IS NULL THEN
        RAISE EXCEPTION 'Value "%" not found in state with activation path %', p_value_name, p_activation_path;
    END IF;

    -- Compute full dependency closure for the target value.
    WITH RECURSIVE unlock_chain(act) AS (
       SELECT v.activation
       FROM value v
       WHERE v.state = v_state_id
         AND v.id = v_value_id
       UNION
       SELECT va.antecedent
       FROM value_antecedent va
       JOIN unlock_chain uc ON va.child = uc.act
       UNION
       SELECT va.child
       FROM value_antecedent va
       JOIN unlock_chain uc ON va.antecedent = uc.act
    )
    -- Delete the locked_dependency rows for this target value.
    DELETE FROM locked_dependency
    WHERE state = v_state_id
      AND value = v_value_id
      AND activation IN (SELECT act FROM unlock_chain);

    -- Now, for each activation in the closure, remove its lock if it’s no longer referenced.
    WITH dependency_set AS (
        SELECT act FROM (
            SELECT DISTINCT act FROM unlock_chain
        ) t
    )
    DELETE FROM locked_activation
    WHERE state = v_state_id
      AND activation IN (
         SELECT act FROM dependency_set
         EXCEPT
         SELECT activation FROM locked_dependency WHERE state = v_state_id
      );
END;
$$;

## Generate State

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE generate_state(
    p_entity_name VARCHAR,
    p_time        DOUBLE PRECISION
)
LANGUAGE plpython3u
AS $$
#
# This implementation uses a generator–based trampoline to preserve immediate child activation
# execution while supporting unmasking (using the stored function check_unmasking) and regeneration.
# All lookups are performed via on–demand SQL queries without in–memory caches.
#

def run_activation(context):
    """
    Run one activation using the context (a dict with keys):
      - mech_id:         mechanism id to run,
      - activation_name: local activation name (None for root),
      - activation_id:   activation id (if already created),
      - root_mech_id:    id of the entity's root mechanism,
      - state_id:        state id,
      - activation_path: full (slash–separated) activation path so far,
      - is_regeneration: boolean flag.
    
    Returns the generator produced by executing the mechanism's main() code.
    """
    current_activation_path = context.get('activation_path', "")

    # --- On–demand regeneration check.
    if context.get('is_regeneration'):
        sql_locked_check = """
            SELECT la.activation 
            FROM locked_activation la 
            WHERE la.state = $1 
              AND get_activation_full_path(la.activation) = $2
            LIMIT 1
        """
        res_locked = plpy.execute(sql_locked_check, [context['state_id'], current_activation_path])
        if res_locked.nrows() > 0:
            # Activation is locked; skip re–execution.
            def empty_gen():
                return
                yield
            return empty_gen()
        # For non–locked activations in regeneration mode, delete previous output rows.
        if context.get('activation_id'):
            plpy.execute(
                "DELETE FROM value WHERE state = $1 AND activation = $2",
                [context['state_id'], context['activation_id']]
            )
    
    # --- On–demand unmasking check for the current activation.
    # For non–root activations, compute parent's activation path and the local name.
    if current_activation_path:
        parent_path, sep, local_name = current_activation_path.rpartition('/')
        if sep == '':
            parent_path = ''
            local_name = current_activation_path
        res_unmask = plpy.execute(
            "SELECT check_unmasking($1, $2, $3) AS unmasked",
            [context['root_mech_id'], parent_path, local_name]
        )
        if res_unmask.nrows() > 0 and res_unmask[0]['unmasked'] is not None:
            context['mech_id'] = res_unmask[0]['unmasked']

    # --- Look up the mechanism record.
    sql = "SELECT id, name, serialized FROM mechanism WHERE id = $1"
    res = plpy.execute(sql, [context['mech_id']])
    if res.nrows() == 0:
        plpy.error("Mechanism with id %s not found" % context['mech_id'])
    mech = res[0]

    # Helper: resolve paths relative to the current activation.
    def resolve_path(input_path, base_path):
        if input_path.startswith("/"):
            return input_path.lstrip("/")
        elif input_path.startswith("./") or input_path.startswith(".."):
            base_components = base_path.split("/") if base_path else []
            resolved_components = list(base_components)
            for part in input_path.split("/"):
                if part in (".", ""):
                    continue
                elif part == "..":
                    if resolved_components:
                        resolved_components.pop()
                    else:
                        plpy.error("Path resolution error: cannot go above the root activation")
                else:
                    resolved_components.append(part)
            return "/".join(resolved_components)
        else:
            return base_path + "/" + input_path if base_path else input_path

    # Inject helper functions for the mechanism code.
    def use_input(path):
        resolved = resolve_path(path, current_activation_path)
        parts = resolved.rsplit("/", 1)
        parent_path = parts[0] if len(parts) > 1 else ""
        output_name = parts[-1]
        sql_val = """
            SELECT v.id AS value_id, v.activation AS antecedent,
                CASE WHEN v.type = 'number'
                        THEN (SELECT serialized FROM number_value WHERE value = v.id)
                        ELSE (SELECT serialized FROM string_value WHERE value = v.id)
                END AS value
            FROM value v
            JOIN activation a ON a.id = v.activation
            WHERE get_activation_full_path(a.id) = $1
            AND v.name = $2
            AND v.state = $3
            LIMIT 1
        """
        res_val = plpy.execute(sql_val, [parent_path, output_name, context['state_id']])
        if res_val.nrows() == 0:
            plpy.error("No output found for resolved path: " + resolved)
        # Record the dependency: current (child) activation used the parent's output.
        plpy.execute(
            "INSERT INTO value_antecedent(value, child, antecedent) VALUES ($1, $2, $3) ON CONFLICT DO NOTHING",
            [res_val[0]['value_id'], context['activation_id'], res_val[0]['antecedent']]
        )
        return res_val[0]['value']

    def add_output(name, value):
        full_output_path = resolve_path(name, current_activation_path)
        sql_dup = """
            SELECT 1 FROM value 
            WHERE state = $1 AND activation = $2 AND name = $3 
            LIMIT 1
        """
        res_dup = plpy.execute(sql_dup, [context['state_id'], context['activation_id'], name])
        if res_dup.nrows() > 0:
            plpy.error("Output with name '%s' already exists in the current activation" % name)
        value_type_val = 'number' if isinstance(value, (int, float)) else 'string'
        sql_ins = """
            INSERT INTO value(state, activation, name, type)
            VALUES ($1, $2, $3, $4)
            RETURNING id
        """
        res_ins = plpy.execute(sql_ins, [context['state_id'], context['activation_id'], name, value_type_val])
        value_id = res_ins[0]['id']
        if value_type_val == 'number':
            plpy.execute("INSERT INTO number_value(value, serialized) VALUES ($1, $2)", [value_id, float(value)])
        else:
            plpy.execute("INSERT INTO string_value(value, serialized) VALUES ($1, $2)", [value_id, str(value)])
        return value

    def activate(mechanism_name, local_activation_name=None):
        if local_activation_name is None:
            local_activation_name = mechanism_name
        new_activation_path = current_activation_path + "/" + local_activation_name if current_activation_path else local_activation_name
        # Check for duplicate activation on–demand.
        sql_check = "SELECT 1 FROM activation WHERE get_activation_full_path(id) = $1 LIMIT 1"
        res_check = plpy.execute(sql_check, [new_activation_path])
        if res_check.nrows() > 0:
            plpy.error("Activation with name '%s' already exists in the current activation" % local_activation_name)
        # Look up the mechanism id for the requested mechanism.
        sql_lookup = "SELECT id FROM mechanism WHERE name = $1 LIMIT 1"
        res_lookup = plpy.execute(sql_lookup, [mechanism_name])
        if res_lookup.nrows() == 0:
            plpy.error("Mechanism with name %s not found" % mechanism_name)
        new_mech_id = res_lookup[0]['id']

        # Insert a new activation record.
        sql_act = """
            INSERT INTO activation(name, from_mechanism, root_mechanism, to_mechanism)
            VALUES ($1, $2, $3, $4)
            RETURNING id
        """
        res_act = plpy.execute(sql_act, [local_activation_name, context['mech_id'], context['root_mech_id'], new_mech_id])
        new_activation_id = res_act[0]['id']
        # Prepare the child context.
        child_context = {
            'mech_id': new_mech_id,
            'activation_name': local_activation_name,
            'activation_id': new_activation_id,
            'root_mech_id': context['root_mech_id'],
            'state_id': context['state_id'],
            'activation_path': new_activation_path,
            'is_regeneration': context.get('is_regeneration', False)
        }

        # On–demand unmasking for the child activation using the stored function.
        res_child_unmask = plpy.execute(
            "SELECT check_unmasking($1, $2, $3) AS unmasked",
            [context['root_mech_id'], current_activation_path, local_activation_name]
        )
        if res_child_unmask.nrows() > 0 and res_child_unmask[0]['unmasked'] is not None:
            child_context['mech_id'] = res_child_unmask[0]['unmasked']
        yield ("activate", child_context)

    def reject(local_activation_name):
        # Compute the full activation path for the activation to be rejected.
        full_path = current_activation_path + "/" + local_activation_name if current_activation_path else local_activation_name

        # Confirm that the activation exists.
        sql_get = "SELECT id FROM activation WHERE get_activation_full_path(id) = $1 LIMIT 1"
        res_get = plpy.execute(sql_get, [full_path])
        if res_get.nrows() == 0:
            plpy.error("Activation with resolved path %s not found" % full_path)
        
        # Build a pattern to match the rejected activation and any descendant activations.
        pattern = full_path + '%'
        
        # Delete only the values (not the activations) for the current state that belong to the activation chain.
        sql_reject = """
            DELETE FROM value
            WHERE state = $1
            AND activation IN (
                    SELECT a.id FROM activation a
                    WHERE get_activation_full_path(a.id) LIKE $2
            )
        """
        plpy.execute(sql_reject, [context['state_id'], pattern])
    
    # Prepare a namespace for the mechanism code.
    local_ns = {
        'use_input': use_input,
        'add_output': add_output,
        'activate': activate,
        'reject': reject,
    }
    exec(mech['serialized'], local_ns)
    if 'main' not in local_ns:
        plpy.error("Mechanism code does not define a main() generator")
    return local_ns['main']()

def trampoline(root_context):
    """
    The trampoline runs activations with immediate semantics by maintaining an explicit
    stack of (context, generator) pairs. When an activation yields an "activate" instruction,
    the child is executed immediately and control returns to the parent when it completes.
    """
    stack = []
    current_gen = run_activation(root_context)
    stack.append((root_context, current_gen))
    while stack:
        current_context, current_gen = stack[-1]
        try:
            instr = next(current_gen)
        except StopIteration:
            stack.pop()
            continue
        if instr[0] == "activate":
            child_context = instr[1]
            child_gen = run_activation(child_context)
            stack.append((child_context, child_gen))
        else:
            plpy.error("Unknown instruction: " + str(instr))

# --- Main Body of generate_state ---

# 1. Look up the entity and its root mechanism.
sql_entity = "SELECT id, mechanism FROM entity WHERE name = $1 LIMIT 1"
res_entity = plpy.execute(sql_entity, [p_entity_name])
if res_entity.nrows() == 0:
    plpy.error("Entity with name '%s' not found" % p_entity_name)
entity_rec = res_entity[0]
entity_id = entity_rec['id']
root_mech_id = entity_rec['mechanism']

# 2. Look up or create the state for the entity at the given time.
sql_state = "SELECT id FROM state WHERE entity = $1 AND time = $2 LIMIT 1"
res_state = plpy.execute(sql_state, [entity_id, p_time])
if res_state.nrows() == 0:
    res_insert = plpy.execute(
        "INSERT INTO state(entity, time) VALUES ($1, $2) RETURNING id", 
        [entity_id, p_time]
    )
    state_id = res_insert[0]['id']
    is_regeneration = False
else:
    state_id = res_state[0]['id']
    is_regeneration = True

# 3. Prepare the root context.
root_context = {
    'mech_id': root_mech_id,
    'activation_name': None,
    'activation_id': None,
    'root_mech_id': root_mech_id,
    'state_id': state_id,
    'activation_path': "",  # The root activation path is empty.
    'is_regeneration': is_regeneration
}

# 4. Run the trampoline to process all activations synchronously.
trampoline(root_context)

$$;

## Get State

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_state(
    p_entity_name VARCHAR,
    p_time DOUBLE PRECISION
)
RETURNS TABLE (
    "index" BIGINT,
    address VARCHAR,
    value_type value_type,
    value TEXT,
    locked BOOLEAN
)
LANGUAGE sql
AS $$
WITH state_info AS (
  SELECT s.id AS state_id, e.mechanism AS root_mech_id, e.name AS entity_name
  FROM state s
  JOIN entity e ON e.id = s.entity
  WHERE e.name = p_entity_name AND s.time = p_time
),
all_values AS (
  SELECT v.*, 
         a.name AS local_activation,
         get_activation_full_path(a.id) AS activation_path
  FROM value v
  JOIN activation a ON a.id = v.activation
  JOIN state_info si ON si.state_id = v.state
)
SELECT 
  row_number() OVER (ORDER BY activation_path) AS "index",
  activation_path || '/' || v.name AS address,
  v.type AS value_type,
  CASE 
    WHEN v.type = 'number' THEN (
         SELECT nv.serialized::text 
         FROM number_value nv 
         WHERE nv.value = v.id
    )
    ELSE (
         SELECT sv.serialized 
         FROM string_value sv 
         WHERE sv.value = v.id
    )
  END AS value,
  EXISTS (
     SELECT 1 
     FROM locked_activation la 
     WHERE la.activation = v.activation AND la.state = v.state
  ) AS locked
FROM all_values v
ORDER BY activation_path;
$$;

## Generate Grouping

In [ ]:
%%sql

CREATE OR REPLACE PROCEDURE generate_grouping(
    p_mechanism_name         VARCHAR,
    p_grouping_name          VARCHAR,
    p_entity_name_template   VARCHAR,
    p_num_entities           INTEGER
)
LANGUAGE plpgsql
AS $$
DECLARE
    v_mech_id      INTEGER;
    v_grouping_id  INTEGER;
    v_entity_id    INTEGER;
    v_idx          INTEGER;
    v_entity_name  VARCHAR;
BEGIN
    -- Look up the mechanism id using the mechanism name.
    SELECT id INTO v_mech_id FROM mechanism WHERE name = p_mechanism_name;
    IF v_mech_id IS NULL THEN
        RAISE EXCEPTION 'No mechanism found with name=%', p_mechanism_name;
    END IF;
    
    -- Create a new grouping.
    INSERT INTO grouping (name)
    VALUES (p_grouping_name)
    RETURNING id INTO v_grouping_id;
    
    FOR v_idx IN 1..p_num_entities LOOP
        v_entity_name := format(p_entity_name_template, v_idx);
        
        -- Create a new entity using the looked-up mechanism id.
        INSERT INTO entity (mechanism, name)
        VALUES (v_mech_id, v_entity_name)
        RETURNING id INTO v_entity_id;
        
        INSERT INTO grouping_entity (entity, grouping)
        VALUES (v_entity_id, v_grouping_id);
        
        -- Call generate_state for the new entity at time 0.
        CALL generate_state(v_entity_name, 0);
    END LOOP;
END;
$$;

## Get Grouping at Time

In [ ]:
%%sql

CREATE OR REPLACE FUNCTION get_grouping_at_time(
    p_grouping_name VARCHAR,
    p_time DOUBLE PRECISION
)
RETURNS TABLE (
    entity_name VARCHAR,
    state_id INTEGER,
    address VARCHAR,
    value_type value_type,
    value TEXT,
    locked BOOLEAN
)
LANGUAGE sql
AS $$
WITH grp AS (
  SELECT id FROM grouping WHERE name = p_grouping_name
),
ent AS (
  SELECT e.id, e.name
  FROM entity e
  JOIN grouping_entity ge ON e.id = ge.entity
  WHERE ge.grouping = (SELECT id FROM grp)
),
st AS (
  SELECT e.id AS entity_id, e.name AS entity_name, s.id AS state_id
  FROM ent e
  JOIN state s ON s.entity = e.id
  WHERE s.time = p_time
)
SELECT 
  st.entity_name,
  st.state_id,
  es.address,
  es.value_type,
  es.value,
  es.locked
FROM st
CROSS JOIN LATERAL (
    SELECT *
    FROM get_state(st.entity_name, p_time)
) es
ORDER BY st.entity_name, es."index";
$$;

# Inspect Schema

## List Tables

In [ ]:
%%sql

SELECT tablename AS name
FROM pg_catalog.pg_tables
WHERE schemaname NOT IN ('pg_catalog', 'information_schema')
ORDER BY schemaname, tablename;

## List Procedures

In [ ]:
%%sql

SELECT proname AS name, pg_catalog.pg_get_function_arguments(p.oid) AS arguments
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'p' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

## List Functions

In [ ]:
%%sql

SELECT proname AS name,
       pg_catalog.pg_get_function_arguments(p.oid) AS arguments,
       pg_catalog.pg_get_function_result(p.oid) AS return_type
FROM pg_catalog.pg_proc p
JOIN pg_catalog.pg_namespace n ON n.oid = p.pronamespace
WHERE p.prokind = 'f' AND n.nspname = 'public'
ORDER BY n.nspname, proname;

## List Indices

In [ ]:
%%sql

SELECT i.relname AS index_name, t.relname AS table_name
FROM pg_catalog.pg_index ix
JOIN pg_catalog.pg_class i ON i.oid = ix.indexrelid
JOIN pg_catalog.pg_class t ON t.oid = ix.indrelid
JOIN pg_catalog.pg_namespace n ON n.oid = t.relnamespace
WHERE n.nspname = 'public'
ORDER BY n.nspname, t.relname, i.relname;

# Testing

## Gender Attribute

### Delete Gender Attribute

In [ ]:
%%sql

CALL delete_discrete_attribute('Gender');

### Add Gender Attribute

In [ ]:
%%sql

call add_discrete_attribute('Gender', '[
    ["Male", 1], 
    ["Female", 1]
]');

### View Gender Attribute

In [ ]:
%%sql

SELECT * FROM get_discrete_attribute('Gender');

## Hair Color Attribute

### Delete Hair Color Attribute

In [ ]:
%%sql

CALL delete_discrete_attribute('Hair Color');

### Add Hair Color Attribute

In [ ]:
%%sql

call add_discrete_attribute('Hair Color', '[
    ["Blonde", 2], 
    ["Brown", 3],
    ["Golden", 2],
    ["Auburn", 2],
    ["Ginger", 1],
    ["Sandy", 2],
    ["Black", 3],
    ["Gray", 2]
]');

### View Hair Color Attribute

In [ ]:
%%sql

SELECT * FROM get_discrete_attribute('Hair Color');

## Height Attribute

### Delete Height Attribute

In [ ]:
%%sql

CALL delete_continuous_attribute('Height');

### Add Height Attribute

In [ ]:
%%sql

call add_continuous_attribute('Height', 0, 68, 96, 32, 0, 4, 'in.', '[
    ["Short", 60],
    ["Average", 74],
    ["Tall", 96]
]');

### View Height Attribute

In [ ]:
%%sql

SELECT * FROM get_continuous_attribute('Height');

## Human Variant

### Delete

In [ ]:
%%sql

CALL delete_variant('Human');

### Add

In [ ]:
%%sql

CALL add_variant('Human', '[
    ["a", "Gender"],
    ["a", "Hair Color"],
    ["a", "Height"]
]');

### Get

In [ ]:
%%sql

SELECT * FROM get_variant('Human', 5);

## Generate a Group of Humans

### Delete

In [ ]:
%%sql

CALL delete_entity_group('Test Humans 1', true);

### Create

In [ ]:
%%sql

CALL generate_entity_group('Human', 'Test Humans 1', 'Test Human %s', 10);

### View

In [ ]:
%%sql

SELECT * FROM get_entity_group_at_time('Test Humans 1', 0);

# The Robot Test

## Paint Attribute

In [ ]:
%%sql

CALL add_discrete_attribute('Color', '[
        ["None", 3],
        ["Red", 2],
        ["Yellow", 2],
        ["Green", 2],
        ["Blue", 2],
        ["Orange", 2],
        ["White", 2],
        ["Black", 2],
        ["Purple", 2],
        ["Custom", 1, "Custom Pigment"]
]');

In [ ]:
%%sql

CALL add_continuous_attribute('R', 0, 127, 255, 0, 0, 0, NULL, NULL);

In [ ]:
%%sql

CALL add_continuous_attribute('G', 0, 127, 255, 0, 0, 0, NULL, NULL);

In [ ]:
%%sql

CALL add_continuous_attribute('B', 0, 127, 255, 0, 0, 0, NULL, NULL);

In [ ]:
%%sql

CALL add_variant('Custom Pigment', '[
    ["a", "R"], 
    ["a", "G"], 
    ["a", "B"]
]');

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Sheen', '[
        ["Glossy", 3],
        ["Egg-Shell", 1],
        ["Matte", 2],
        ["Semi-Gloss", 2]
]');

In [ ]:
%%sql

CALL add_variant('Paint', '[
    ["a", "Color"], 
    ["a", "Sheen"]
]');

## Material Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Material', '[
        ["Steel", 4],
        ["Aluminum", 6],
        ["Bronze", 2],
        ["Iron", 2],
        ["Carbon-Fiber", 6],
        ["PVC", 4],
        ["Animal Bone", 1],
        ["Wood", 2]
    ]'
);

## Power-Level Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Power-Level', 0, 50, 100, 24, 0, 2, NULL, '[
    ["Very Weak", 20],
    ["Weak", 40],
    ["Normal", 60],
    ["Strong", 80],
    ["Ultra", 100]
]');

## Length Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Length', .01, 36, 120, 36, 0, 2, 'in.', NULL);

## Diameter Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Diameter', .01, 1, 4, 12, 0, 2, 'in.', NULL);

## Sub-Arm Attribute

In [ ]:
%%sql

CALL add_discrete_attribute('Sub-Arm', '[["No", 4], ["Yes", 1, "Arm"]]');

## Attachment Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Attachment', '[
        ["Cutter", 3],
        ["Welder", 2],
        ["Pincher", 3],
        ["Grabber", 3],
        ["Laser", 1],
        ["Cannon", 1],
        ["Hammer", 2],
        ["Grappler", 1],
        ["Vacuum", 2]
    ]'
);

## Input Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Input', '[
        ["Camera", 5],
        ["Microphone", 5],
        ["Smell-O-Sense", 2],
        ["Infra-Red", 3],
        ["X-Ray", 3],
        ["GPS", 4],
        ["Temporal", 1]
    ]'
);

## Output Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Output', '[
        ["MIDI Tone", 5],
        ["Text Screen", 7],
        ["Clicker", 4],
        ["Recorded Phrases", 7],
        ["X-Ray", 5],
        ["Light Array", 5],
        ["Party Blower", 1],
        ["Kazoo", 1]
    ]'
);

## Foot Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Foot', '[
        ["Roller Skate", 5],
        ["Stomper", 7],
        ["Tread", 4],
        ["Pogo", 7],
        ["Thruster", 5],
        ["Suction", 5]
    ]'
);

## Shape Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Shape', '[
        ["Spherical", 5],
        ["Blocky", 7],
        ["Sculpted", 4],
        ["Spiky", 2],
        ["Segmented", 5],
        ["Donut", 1]
    ]'
);

## Power Source Attribute

In [ ]:
%%sql

CALL add_discrete_attribute(
    'Power Source', '[
        ["Battery", 15],
        ["Solar", 10],
        ["Diesel", 10],
        ["Wired", 12],
        ["Wind", 4],
        ["Nuclear Core", 4]
    ]'
);

## Neck Width Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Neck Width', 1, 10, 2, 2, 0, 1, 'in.', NULL);

## Weight Attribute

In [ ]:
%%sql

CALL add_continuous_attribute('Weight', .01, 20, 5000, 2, 0, 1, 'lbs.', NULL);

## Torso Variant

In [ ]:
%%sql

CALL add_variant('Torso', '[
    ["a", "Shape"], 
    ["a", "Material"], 
    ["a", "Power-Level"], 
    ["v", "Paint"]
]');

## Head Variant

In [ ]:
%%sql

CALL add_variant('Head', '[
    ["a", "Input"], 
    ["a", "Output"], 
    ["a", "Material"], 
    ["a", "Power-Level"], 
    ["v", "Paint"]
]');

## Leg Variant

In [ ]:
%%sql

CALL add_variant('Leg', '[
    ["a", "Foot"], 
    ["a", "Material"], 
    ["a", "Power-Level"], 
    ["a", "Length"],
    ["a", "Diameter"],
    ["v", "Paint"]
]');

## Hand Variant

In [ ]:
%%sql

-- Create the variant groupings
CALL add_variant('Hand', '[
    ["a", "Attachment"], 
    ["a", "Material"],  
    ["a", "Power-Level"], 
    ["v", "Paint"]
]');

## Arm Variant

In [ ]:
%%sql

CALL add_variant('Arm', '[
    ["v", "Hand"], 
    ["a", "Material"], 
    ["v", "Paint"], 
    ["a", "Length"], 
    ["a", "Diameter"], 
    ["a", "Sub-Arm"]
]');

## Robot Variant

In [ ]:
%%sql

CALL add_variant('Robot', '[
    ["a", "Power Source"],
    ["v", "Head"],
    ["a", "Neck Width"],
    ["v", "Torso"],
    ["v", "Arm", "Left Arm"], 
    ["v", "Arm", "Right Arm"], 
    ["v", "Leg", "Left Leg"], 
    ["v", "Leg", "Right Leg"],
    ["a", "Weight"]
]');

## Effects

### Welder hands cannot be made of wood'

In [ ]:
%%sql

-- Add effects

-- Test: discrete effect on same variant level, disable span
CALL add_discrete_effect(
    'Welder hands cannot be made of wood',
    'Hand',
    '[["Attachment", "Welder"]]',
    '["Material"]',
    '[["Wood", 0]]'
);

### Wooden hands have lower power level

In [ ]:
%%sql

-- Test: continous effect on same variant level
CALL add_continuous_effect(
    'Wooden hands have lower power level',
    'Hand',
    '[["Material", "Wood"]]',
    '["Power-Level"]',
    -20, 0, 0
);

### Spherical torsos are more likely to be painted white or blue

In [ ]:
%%sql

-- Test: stacked discrete effect on an attribute one level down
CALL add_discrete_effect(
    'Spherical torsos are more likely to be painted white or blue',
    'Torso',
    '[["Shape", "Spherical"]]',
    '["Paint", "Color"]',
    '[["White", 10], ["Blue", 5]]'
);

### Animal bone torsos are more likely to be painted white or green

In [ ]:
%%sql

CALL add_discrete_effect(
    'Animal bone torsos are more likely to be painted white or green',
    'Torso',
    '[["Material", "Animal Bone"]]',
    '["Paint", "Color"]',
    '[["White", 10], ["Green", 5]]'
);

### Wooden torsos have lower power level

In [ ]:
%%sql

-- Test: stacked continuous effect on same variant level
CALL add_continuous_effect(
    'Wooden torsos have lower power level',
    'Torso',
    '[["Material", "Wood"]]',
    '["Power-Level"]',
    -20, 0, 0
);

### Blocky torsos have higher power level

In [ ]:
%%sql

CALL add_continuous_effect(
    'Blocky torsos have higher power level',
    'Torso',
    '[["Shape", "Blocky"]]',
    '["Power-Level"]',
    15, 0, 0
);

### All diesel-powered robots have a roller skate on their left foot

In [ ]:
%%sql

-- Test: discrete effect on an attribute one level down, restricting options
CALL add_discrete_effect(
    'All diesel-powered robots have a roller skate on their left foot',
    'Robot',
    '[["Power Source", "Diesel"]]',
    '["Left Leg", "Foot"]',	
    '[["Roller Skate", 1]]',
    true
);

### Robot with wired power are less likely to have heads with camera input

In [ ]:
%%sql

-- Test: one level down
CALL add_discrete_effect(
    'Wired-powered robots are less likely to have camera-input heads',
    'Robot',
    '[["Power Source", "Wired"]]',
    '["Head", "Input"]',	
    '[["Camera", -3]]',
    true
);

### Robots with camera input heads are more likely to have wider necks

In [ ]:
%%sql

-- Test: continuous effect on an attribute multiple levels down (including recursive)
CALL add_continuous_effect(
    'Robots with camera input heads are more likely to have wider necks',
    'Robot',
    '["Head", ["Input", "Camera"]]',
    '["Neck Width"]',
    5, 0, 0
);

### Robots with nuclear cores have high-powered heads

In [ ]:
%%sql

-- Test: continuous effect on an attribute multiple levels down (including recursive)
CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered heads',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Head", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered torsos

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered torsos',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Torso", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered left legs

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered left legs',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Left Leg", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered right legs

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered right legs',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Right Leg", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered right hands

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered right hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Right Arm", "Hand", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered left hands

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered left hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Left Arm", "Hand", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered right sub-arm hands

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered right sub-arm hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Right Arm", ["Sub-Arm", "Yes"], "Hand", "Power-Level"]',
    90, 6, 4
);

### Robots with nuclear cores have high-powered left sub-arm hands

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with nuclear cores have high-powered left sub-arm hands',
    'Robot',
    '[["Power Source", "Nuclear Core"]]',
    '["Left Arm", ["Sub-Arm", "Yes"], "Hand", "Power-Level"]',
    90, 6, 4
);

### Robots with steel torsos have a higher weight

In [ ]:
%%sql

-- Test: continous effect on an attribute one level up
CALL add_continuous_effect(
    'Robots with steel torsos have a higher weight',
    'Robot',
    '["Torso", ["Material", "Steel"]]',
    '["Weight"]',
    50, 0, 2
);

### Robots with iron torsos have a higher weight

In [ ]:
%%sql

CALL add_continuous_effect(
    'Robots with iron torsos have a higher weight',
    'Robot',
    '["Torso", ["Material", "Iron"]]',
    '["Weight"]',
    100, 0, 3
);

### Grappler hands are more likely to be on metal arms

In [ ]:
%%sql

-- Test: discrete effect on an attribute one level up
CALL add_discrete_effect(
    'Grappler hands are more likely to be on metal arms',
    'Arm',
    '["Hand", ["Attachment", "Grappler"]]',
    '["Material"]',
    '[["Steel", 5],
      ["Aluminum", 5],
      ["Bronze", 5],
      ["Iron", 5],
      ["Carbon-Fiber", 5]]',
    true
);

### Heads with kazoo outputs can be painted rainbow

In [ ]:
%%sql

-- Test: discrete effect that adds new spans
CALL add_discrete_effect(
    'Heads with kazoo outputs can be painted rainbow',
    'Head',
    '[["Output", "Kazoo"]]',
    '["Paint", "Color"]',
    '[["Rainbow", 100]]'
);

### Get Variant

In [ ]:
%%sql

SELECT * FROM get_variant('Robot', 4);

### Generate Robots

In [ ]:
%%sql

CALL generate_entity_group('Robot', 'Test Robots 1', 'Robot %s', 10);

### Inspect Output

In [ ]:
%%sql

SELECT * FROM get_entity_group_at_time('Test Robots 1', 0);